In [1]:
from autopycoin.data import random_ts
from autopycoin.models import PoolNBEATS, create_interpretable_nbeats
from autopycoin.dataset import WindowGenerator
import tensorflow as tf
import pandas as pd

data = random_ts(n_steps=1000,
                 trend_degree=2,
                 periods=[10],
                 fourier_orders=[10],
                 trend_mean=0,
                 trend_std=1,
                 seasonality_mean=0,
                 seasonality_std=1,
                 batch_size=1,
                 n_variables=1,
                 noise=True,
                 seed=42)
data = pd.DataFrame(data[0].numpy(), columns=['test'])

w = WindowGenerator(
       input_width=70,
       label_width=10,
       shift=10,
       test_size=50,
       valid_size=10,
       flat=True,
       batch_size=32,
       preprocessing=lambda x,y: (x, (x, y))
    )

w = w.from_array(data=data,
       input_columns=['test'],
       label_columns=['test'])

nbeats_model = lambda label_width: create_interpretable_nbeats(label_width=label_width, 
seasonality_n_neurons=20, trend_n_neurons=20)

model = PoolNBEATS(
            label_width=10,
            n_models=1,
            nbeats_models=nbeats_model,
            )
model.compile(tf.keras.optimizers.Adam(
   learning_rate=0.0015, beta_1=0.9, beta_2=0.999, epsilon=1e-07, amsgrad=True,
   name='Adam'), loss=['mse', 'mae'], metrics=['mae'])
history = model.fit(w.train, validation_data=w.valid, epochs=1, verbose=1)

model.predict(w.test.take(1))

27/27 [==============================] - 9s 119ms/step - loss: 82.1109 - output_1_1_loss: 82.1109 - output_1_2_loss: 0.0000e+00 - output_1_1_mae: 6.2152 - output_1_2_mae: 3.6925 - val_loss: 24.4680 - val_output_1_1_loss: 24.4680 - val_output_1_2_loss: 0.0000e+00 - val_output_1_1_mae: 3.7548 - val_output_1_2_mae: 2.7131


([UnivariateTensor(values=<tf.Tensor: shape=(32, 40), dtype=float32, numpy=
  array([[-0.5245755 , -0.44177097, -0.09150481, ..., -4.4183493 ,
          -0.43652332, -0.60034466],
         [-1.0522995 ,  2.788721  ,  2.5973752 , ..., -7.2079887 ,
          -6.1845384 , -4.4375777 ],
         [-2.1808124 ,  1.1238884 , -0.46742713, ..., -3.1518397 ,
           1.2203534 ,  1.7466216 ],
         ...,
         [-3.8898344 , -0.29096508,  3.0282083 , ..., -5.619008  ,
           1.753809  ,  2.378242  ],
         [-0.27902007,  1.7618431 , -0.09417248, ..., -2.1943598 ,
          -1.1873442 ,  0.06524181],
         [ 1.0937445 ,  1.0216615 ,  0.12289441, ..., -6.6032662 ,
          -7.848634  , -3.4875789 ]], dtype=float32)>, quantiles=False, shape=TensorShape([32, 40]), dtype=tf.float32, multivariates=False)],
 array([[-0.85373557,  0.05985489, -0.95102006, -0.27060872, -0.6299182 ,
         -1.6519103 , -0.7383224 , -1.7491983 , -1.0687846 , -1.4280967 ],
        [ 2.099026  , -0.2610333

In [2]:
model.predict(w.test.take(1))[1]

array([[-0.85373557,  0.05985489, -0.95102006, -0.27060872, -0.6299182 ,
        -1.6519103 , -0.7383224 , -1.7491983 , -1.0687846 , -1.4280967 ],
       [ 2.099026  , -0.2610333 ,  3.9747052 ,  2.3996272 ,  1.2362671 ,
         3.3424454 ,  0.9823828 ,  5.2181225 ,  3.6430452 ,  2.4796848 ],
       [ 0.1662855 ,  1.2600462 ,  2.188526  ,  0.28159884, -0.88415277,
         0.01934171,  1.113101  ,  2.0415785 ,  0.13465446, -1.0310977 ],
       [-2.260236  ,  3.9043083 , -2.3878126 , -0.08810949,  0.5255131 ,
        -2.0463228 ,  4.1182218 , -2.1739013 ,  0.12580203,  0.73941886],
       [-2.389771  , -0.8310424 ,  3.6241693 ,  0.5664227 , -2.970701  ,
        -1.871111  , -0.3123857 ,  4.1428246 ,  1.0850804 , -2.4520447 ],
       [-0.3650172 , -1.0631263 ,  0.5141908 , -1.4352435 , -1.4369147 ,
        -0.37641576, -1.0745265 ,  0.502789  , -1.4466436 , -1.4483145 ],
       [-0.20211169,  1.6179272 , -0.5165374 , -0.08837366, -0.6034236 ,
        -0.7546034 ,  1.0654349 , -1.0690378 

In [ ]:
        # aggregation if possible
        if all(inputs_reconstructed[0].shape == tensor.shape for tensor in inputs_reconstructed[1:]):
            inputs_reconstructed = self._fn_agg(inputs_reconstructed, axis=-1)

        # aggregation if possible
        if all(outputs[0].shape == tensor.shape for tensor in outputs[1:]):
            outputs = self._fn_agg(outputs, axis=-1)